<a href="https://colab.research.google.com/github/GuAparecido/Projeto_Final_Python/blob/main/ProjetoFinalPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import requests
import time
import hashlib
from google.colab import userdata

In [ ]:
# Request Url: http://gateway.marvel.com/v1/public/comics
# Request Method: GET
urlComics = "http://gateway.marvel.com/v1/public/comics"
urlCharacter = "http://gateway.marvel.com/v1/public/characters"
urlEvents = "http://gateway.marvel.com/v1/public/events"

# Declarando as variáveis necessárias
api_key = userdata.get('api_key')
ts = str(int(time.time()))
secret_key = userdata.get('secret_key')
string_to_hash =  ts + secret_key + api_key
hash_value = hashlib.md5(string_to_hash.encode('utf-8')).hexdigest()

#Definindo parâmetros de utilização da API.
params = {
    "apikey": api_key,
    "ts": ts,
    "hash": hash_value
}

# Buscando requisições dos quadrinhos
responseComics = requests.get(urlComics, params=params)

# Buscando requisições dos personagens
responseCharacters = requests.get(urlCharacter, params=params)

# # Buscando requisições dos eventos
responseEvents = requests.get(urlEvents, params=params)

# Validando o retorno das APIs
# Quadrinhos
if responseComics.status_code == 200:
    dadosComics = responseComics.json()
    print("Tudo certo com a requisição de comics")
else:
    print(f"Erro na requisição: {responseComics.status_code}")

# Personagens
if responseCharacters.status_code == 200:
    dadosCharacters = responseCharacters.json()
    print("Tudo certo com a requisição de characters")
else:
    print(f"Erro na requisição: {responseCharacters.status_code}")

# Eventos
if responseEvents.status_code == 200:
    dadosEvents = responseEvents.json()
    print("Tudo certo com a requisição de events")
else:
    print(f"Erro na requisição: {responseEvents.status_code}")

In [ ]:
# Verificando o retorno
dadosComics

In [ ]:
# Verificando o retorno
dadosCharacters

In [ ]:
# Verificando o retorno
dadosEvents

In [ ]:
# Validando status
statusComics = dadosComics.get('status')
print(statusComics)

statusCharacters = dadosCharacters.get('status')
print(statusCharacters)

statusEvents = dadosEvents.get('status')
print(statusEvents)

In [ ]:
# Encurtando o caminho aos objetos desejados
resultsComics = dadosComics.get('data', {}).get('results', [])
resultsComics

In [ ]:
# Encurtando o caminho aos objetos
resultsCharacters = dadosCharacters.get('data', {}).get('results', [])
resultsCharacters

In [ ]:
# Encurtando o caminho aos objetos
resultsEvents = dadosEvents.get('data', {}).get('results', [])
resultsEvents